In [ ]:
### code taken from https://dev.to/m0nica/how-to-use-the-tmdb-api-to-find-films-with-the-highest-revenue-82p

import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis
import seaborn as sns
import glob as glob

api_key = '64bc8db1ac9143a7dbb820d7d420ac96'

scrape = False

In [ ]:
if scrape = True:

    results = []

    for year in [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:

        page_number = 1

        response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' 
                                 +  api_key + '&page=' + str(page_number) + 
                                 '&with_original_language=en&primary_release_year=' + str(year) + '&with_release_type=3&with_runtime.gte=80')

        total_pages = response.json()['total_pages']
        # total_pages = 5

        results.extend(response.json()['results'])

        while page_number < total_pages:

            page_number += 1

            response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' 
                                 +  api_key + '&page=' + str(page_number) + 
                                 '&with_original_language=en&primary_release_year=' + str(year) + '&with_release_type=3&with_runtime.gte=80')

            results.extend(response.json()['results'])
            
else: print('Scrape is set to False.')

In [ ]:
pip install tmdbsimple
import tmdbsimple as tmdb
tmdb.API_KEY = '64bc8db1ac9143a7dbb820d7d420ac96'

In [ ]:
df = pd.DataFrame(results)

df.head()

In [ ]:
if scrape = True:

    df_ids = df['id']

    full_movies = []

    for idx in df_ids:
        movie = tmdb.Movies(idx)

        movie_dic = {}

        movie_dic.update(movie.info())

        movie.releases()
        for c in movie.countries:
            if c['iso_3166_1'] == 'US':
                certification = c['certification']
    #         else:
    #             certification = 'None'

        movie_dic.update({'mpaa_rating' : str(certification)})

        full_movies.append(movie_dic)

    df_full = pd.DataFrame(full_movies)

    df_full.to_csv('2012-2019 FULL.csv')
    
else: print('Scrape is set to False.')
    
#### 293471 & 437584 didn't work, make sure to write how i worked around

In [ ]:
#create function later
# https://www.youtube.com/watch?v=ox_AHkeqNJ0&ab_channel=AbhishekAgarrwal

df['production_budget'] = pd.to_numeric(df['production_budget'].replace('[^0-9\.-]', '', regex=True))
df['domestic_gross'] = pd.to_numeric(df['domestic_gross'].replace('[^0-9\.-]', '', regex=True))
df['worldwide_gross'] = pd.to_numeric(df['worldwide_gross'].replace('[^0-9\.-]', '', regex=True))

In [ ]:
df['gross_profit'] = df['worldwide_gross'] - df['production_budget']

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df['production_budget'], df['gross_profit'])

ax.xaxis.set_major_formatter('${x:1.0f}')
ax.xaxis.set_tick_params(which='major', labelcolor='green',
                         labelleft=True, labelright=False)
ax.yaxis.set_major_formatter('${x:1.0f}')
ax.yaxis.set_tick_params(which='major', labelcolor='green',
                         labelleft=True, labelright=False)

In [ ]:
sns.lmplot(data = df_m, x = "production_budget", y = "net_profit")
plt.show()

In [ ]:
response = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=' + api_key + '&language=en-US')

# Action          28
# Adventure       12
# Animation       16
# Comedy          35
# Crime           80
# Documentary     99
# Drama           18
# Family          10751
# Fantasy         14
# History         36
# Horror          27
# Music           10402
# Mystery         9648
# Romance         10749
# Science Fiction 878
# TV Movie        10770
# Thriller        53
# War             10752
# Western         37

# print(response.json()['genres'])
print(response.json()['genres'][0])


# case_list = {}
# for entry in entries_list:
#     case = {'key1': value, 'key2': value, 'key3':value }
#     case_list.update(case)

genre_dict = {}
all_genres = []

for genre in response.json()['genres']:
    genre_add = {genre['id'] : genre['name']}
    genre_dict.update(genre_add)
    all_genres.append(genre['name'])
print(genre_dict)
print(all_genres)

In [ ]:
genre_df = df

In [ ]:
genre_list = []

for row in genre_df['genre_ids']:
    if row == '[]':
        row = ['None']
        genre_list.append(row)
    else:
        # https://stackoverflow.com/questions/19334374/python-converting-a-string-of-numbers-into-a-list-of-int
        row = [int(s) for s in str(row).replace('[', '').replace(']', '').split(', ')] 
        row = [genre_dict[x] for x in row]

        
        genre_list.append(row)
genre_list

In [ ]:
df.explode('genre_ids').groupby('genre_ids').mean().round()

In [ ]:
# https://futurestud.io/tutorials/matplotlib-stacked-bar-plots
# the first call is as usual
genre_name = all_genre_df['genre_name']
production_budget_avg = all_genre_df['production_budget_avg']
gross_profit_avg = all_genre_df['gross_profit_avg']

plt.figure(figsize = (15,5))
plt.barh(genre_name, production_budget_avg, color="#6c3376") #average profit

# the second one is special to create stacked bar plots
plt.barh(genre_name, gross_profit_avg, left=production_budget_avg, color="#f3e151")

plt.xlabel('Year')  
plt.ylabel('Number of futurestud.io Tutorials')

plt.show()  

In [ ]:
df['date'] = df['date'].astype("datetime64")
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
#james in class

df['year'].plot(kind="hist")

sns.countplot(data = df, x = 'year')



In [ ]:
293471 & 437584